In [1]:
import re
def normalize_text(text, process_multiple_texts=False):
    if process_multiple_texts:
        text = ' '.join(text)
    # Remove any empty space that are placed before special characters excpet for /, (, [, {
    text = re.sub(r'\s+([^\w\s(\[\{])', r'\1', text)
    
    # Remove any empty space that are placed after the character /,(, [, {
    text = re.sub(r'([/(\[\{])\s+', r'\1', text)
    return text

def remove_special_characters(text):
    text = text.lower().replace(' ', '')
    return re.sub(r'[^a-zA-Z0-9\s]', '', text)

In [2]:
import os
import re
import numpy as np
import pandas as pd

train_dataset = pd.DataFrame(columns=['text', 'tokenized_text', 'label', 'unique_label'])
master_path = 'en'    
    
for dataset_name in os.listdir(master_path):
    rows = {}
    all_domain_terms = []
    all_tokenized_domain_terms =[]
    all_domain_term_positions = []
    all_candidate_words = []
    all_texts = []
    all_term_types = []
    all_tokenized_texts = []
    dataset_path = os.path.join(master_path, dataset_name, 'annotated')
    
    if dataset_name == 'cor':
        continue
    
    for category in sorted(os.listdir(dataset_path)):
        category_path = os.path.join(dataset_path, category)
        
        if category == 'annotations':
            for seq_annotation in sorted(os.listdir(category_path)):                                
                if seq_annotation == 'sequential_annotations':
                    iob_annotation_path = os.path.join(category_path, seq_annotation, 'iob_annotations', 'with_named_entities')
                    for filename in sorted(os.listdir(iob_annotation_path), key=lambda x: int(os.path.splitext(x)[0].split('_')[2])):
                        file_path = os.path.join(iob_annotation_path, filename)
                        lines = pd.read_csv(file_path,  sep='\t', na_values=[], skip_blank_lines=False, names=['word', 'boi'])
                    
                        split_indexes = lines[pd.isna(lines[['word', 'boi']]).all(axis=1)].index
                        for prev_index, split_index in zip([-1]+list(split_indexes[:-1]), split_indexes):
                            split_lines = lines.iloc[prev_index+1:split_index]
                            tokenized_domain_terms = []
                            domain_term_positions = []
                            domain_term_position = []
                            domain_terms = []
                            domain_term = []
                            candidate_words = []
                            for i in range(len(split_lines)):
                                candidate_word = split_lines.iloc[i]['word']                                    
                                boi = split_lines.iloc[i]['boi']
                                if candidate_word is np.nan and boi is not np.nan:
                                    candidate_word = 'None'
                                
                                if boi == 'B':
                                    if len(domain_term) == 0:
                                        domain_term.append(candidate_word)
                                        domain_term_position.append(i)
                                    else:
                                        tokenized_domain_terms.append(domain_term)
                                        domain_term = normalize_text(domain_term, True)
                                        domain_terms.append(domain_term)
                                        domain_term = [candidate_word]
                                        
                                        domain_term_position.append(i)
                                        domain_term_positions.append(domain_term_position)
                                        domain_term_position = [i]
                                elif boi == 'I':
                                    domain_term.append(candidate_word)
                                else:
                                    if len(domain_term) > 0:
                                        tokenized_domain_terms.append(domain_term)
                                        domain_term = normalize_text(domain_term, True)
                                        domain_terms.append(domain_term)
                                        domain_term = []
                                        
                                        domain_term_position.append(i)
                                        domain_term_positions.append(domain_term_position)
                                        domain_term_position=[]
                                                                                
                                candidate_words.append(candidate_word.lower())
                            
                            if len(domain_term) > 0:
                                tokenized_domain_terms.append(domain_term)
                                domain_term = normalize_text(domain_term, True)
                                domain_terms.append(domain_term)
                                
                                domain_term_position.append(i)
                                domain_term_positions.append(domain_term_position)
                                                                    
                            all_domain_terms.append(domain_terms)
                            all_tokenized_domain_terms.append(tokenized_domain_terms)
                            all_domain_term_positions.append(domain_term_positions)
                            # all_candidate_words.append(candidate_words)
                            all_candidate_words.append(remove_special_characters(''.join(candidate_words)))
                            
                elif seq_annotation == 'unique_annotation_lists':
                    unique_annotation_list_path = os.path.join(category_path, seq_annotation, f"{dataset_name}_en_terms_nes.tsv")
                    lines = pd.read_csv(unique_annotation_list_path,  sep='\t', names=['word', 'term_type'])
                    for i, domain_terms in enumerate(all_domain_terms):
                        term_types = []
                        for domain_term in domain_terms:
                            domain_term = domain_term.lower()
                            term_type = ''
                            count = 0
                            while len(term_type) == 0:                                    
                                if count==0:
                                    term_type = lines[lines['word']==domain_term]['term_type']
                                
                                elif count==1:
                                    character_removed_domain_term_wo_ws = re.sub(r'[^a-zA-Z0-9\s]', '', domain_term)
                                    term_type = lines[lines['word']==character_removed_domain_term_wo_ws]['term_type']
                                    
                                elif count==2:
                                    character_removed_domain_term_w_ws = re.sub(r'[^a-zA-Z0-9\s]', ' ', domain_term).split(' ')
                                    for i in sorted(range(len(character_removed_domain_term_w_ws)), reverse=True):
                                        for j in range(len(character_removed_domain_term_w_ws)):
                                            sliced_dash_removed_domain_term_w_ws = ' '.join(character_removed_domain_term_w_ws[j:j+i])
                                            term_type = lines[lines['word'].str.contains(sliced_dash_removed_domain_term_w_ws)]['term_type']
                                            if len(term_type) > 0:
                                                break
                                        if len(term_type) > 0:
                                            break
                                
                                elif count > 2:
                                    term_type = 'Not Annotated'                                            
                                    print(f'Path: {unique_annotation_list_path}, domain_term: {domain_term}, index: {i}')
                                    break
                                    
                                count += 1                                    

                            if not isinstance(term_type, str):
                                term_type = term_type.astype(str).values[0]
                            term_types.append(term_type)
                        all_term_types.append(term_types)
        

        elif category == 'texts_tokenised':
            i =0
            for filename in sorted(os.listdir(category_path), key=lambda x: int(os.path.splitext(x)[0].split('_')[2])):
                print(f'###### Filename: {filename} is processing... #######')
                with open(os.path.join(category_path, filename), 'r') as f:
                    tokenized_text = f.read().split('\n')
                    text = list(map(lambda x: normalize_text(x.strip(), False), tokenized_text))
                    if text[-1] == '':
                        text = text[:-1]
                    all_texts.extend(text)
                    # remove empty list from tokenized_text
                    tokenized_text = list(filter(lambda x: x != '', tokenized_text))
                    all_tokenized_texts.extend(list(map(lambda x: x.split(' '), tokenized_text)))
                print(f'###### Filename: {filename} is finished! #######\n')
    
    # Check if doamin terms are correctly extracted
    for texts, domain_terms in zip(all_texts, all_domain_terms):
        for domain_term in domain_terms:
            if domain_term not in texts:
                raise ValueError(f'{domain_term} is not in {texts}')
            
    rows = {'text': all_texts, 'tokenized_text': all_tokenized_texts, 'label': all_domain_terms, 'tokenized_label':all_tokenized_domain_terms, 'all_domain_term_positions':all_domain_term_positions, 'unique_label': all_term_types}
    rows['dataset_name'] = [dataset_name]*len(all_texts)
    if dataset_name == 'htfl':
        rows['domain'] = ['Heart Failure']*len(all_texts)
        test_dataset = pd.DataFrame(rows)
    elif dataset_name == 'equi':
        rows['domain'] = ['Equitation']*len(all_texts)
        validation_dataset = pd.DataFrame(rows)
    else:
        dataset_name_to_domain = {'corp': 'Corruption', 'wind': 'Wind Energy'}
        rows['domain'] = [dataset_name_to_domain[dataset_name]]*len(all_texts)
        train_dataset = pd.concat([train_dataset, pd.DataFrame(rows)])
        

###### Filename: wind_en_01.txt is processing... #######
###### Filename: wind_en_01.txt is finished! #######

###### Filename: wind_en_02.txt is processing... #######
###### Filename: wind_en_02.txt is finished! #######

###### Filename: wind_en_04.txt is processing... #######
###### Filename: wind_en_04.txt is finished! #######

###### Filename: wind_en_26.txt is processing... #######
###### Filename: wind_en_26.txt is finished! #######

###### Filename: wind_en_32.txt is processing... #######
###### Filename: wind_en_32.txt is finished! #######

###### Filename: corp_en_01.txt is processing... #######
###### Filename: corp_en_01.txt is finished! #######

###### Filename: corp_en_02.txt is processing... #######
###### Filename: corp_en_02.txt is finished! #######

###### Filename: corp_en_04.txt is processing... #######
###### Filename: corp_en_04.txt is finished! #######

###### Filename: corp_en_05.txt is processing... #######
###### Filename: corp_en_05.txt is finished! #######

#

In [3]:
print('BEFORE FILTERING:\n')
print(f'Train dataset length: {len(train_dataset)}')
print(f'Validation dataset length: {len(validation_dataset)}')
print(f'Test dataset length: {len(test_dataset)}')

BEFORE FILTERING:

Train dataset length: 8640
Validation dataset length: 3090
Test dataset length: 2432


In [4]:
train_dataset = train_dataset[train_dataset['text'].apply(lambda x: len(x) > 2)]
validation_dataset = validation_dataset[validation_dataset['text'].apply(lambda x: len(x) > 2)]
test_dataset = test_dataset[test_dataset['text'].apply(lambda x: len(x) > 2)]

In [5]:
print('AFTER FILTERING:\n')
print(f'Train dataset length: {len(train_dataset)}')
print(f'Validation dataset length: {len(validation_dataset)}')
print(f'Test dataset length: {len(test_dataset)}')

AFTER FILTERING:

Train dataset length: 4820
Validation dataset length: 3006
Test dataset length: 2432


In [6]:
pd.set_option('display.max_colwidth', 500)
test_dataset.head(15)

,text,tokenized_text,label,tokenized_label,all_domain_term_positions,unique_label,dataset_name,domain
0,"National trends in patient safety for four common conditions, 2005- 2011.","[National, trends, in, patient, safety, for, four, common, conditions, ,, 2005, -, 2011, ., ]","[patient, conditions]","[[patient], [conditions]]","[[3, 4], [8, 9]]","[Common_Term, Common_Term]",htfl,Heart Failure
1,The effects of more than a decade of national efforts dedicated to improve patient safety remain largely unclear.,"[The, effects, of, more, than, a, decade, of, national, efforts, dedicated, to, improve, patient, safety, remain, largely, unclear, ., ]",[patient],[[patient]],"[[13, 14]]",[Common_Term],htfl,Heart Failure
2,"This study used the Medicare Patient Safety Monitoring System (MPSMS) database to assess national trends in adverse event rates between 2005 through 2011 for patients hospitalized with acute myocardial infarction, congestive heart failure, pneumonia, or conditions requiring surgery.","[This, study, used, the, Medicare, Patient, Safety, Monitoring, System, (, MPSMS, ), database, to, assess, national, trends, in, adverse, event, rates, between, 2005, through, 2011, for, patients, hospitalized, with, acute, myocardial, infarction, ,, congestive, heart, failure, ,, pneumonia, ,, or, conditions, requiring, surgery, ., ]","[Medicare Patient Safety Monitoring System, MPSMS, adverse event, patients, hospitalized, acute myocardial infarction, congestive heart failure, pneumonia, conditions, surgery]","[[Medicare, Patient, Safety, Monitoring, System], [MPSMS], [adverse, event], [patients], [hospitalized], [acute, myocardial, infarction], [congestive, heart, failure], [pneumonia], [conditions], [surgery]]","[[4, 9], [10, 11], [18, 20], [26, 27], [27, 28], [29, 32], [33, 36], [37, 38], [40, 41], [42, 43]]","[Named_Entity, Named_Entity, Specific_Term, Common_Term, Common_Term, Specific_Term, Specific_Term, Common_Term, Common_Term, Common_Term]",htfl,Heart Failure
3,"The analysis included a large study sample with more than 60,000 patients across 4372 hospitals.","[The, analysis, included, a, large, study, sample, with, more, than, 60,000, patients, across, 4372, hospitals, ., ]","[patients, hospitals]","[[patients], [hospitals]]","[[11, 12], [14, 15]]","[Common_Term, Common_Term]",htfl,Heart Failure
4,"The results show a significant decline in adverse event rates for acute myocardial infarction and congestive heart failure, translating to an estimated 81,000 in-hospital adverse events averted in 2010–2011.","[The, results, show, a, significant, decline, in, adverse, event, rates, for, acute, myocardial, infarction, and, congestive, heart, failure, ,, translating, to, an, estimated, 81,000, in-hospital, adverse, events, averted, in, 2010–2011, ., ]","[significant, adverse event, acute myocardial infarction, congestive heart failure, in-hospital, adverse events]","[[significant], [adverse, event], [acute, myocardial, infarction], [congestive, heart, failure], [in-hospital], [adverse, events]]","[[4, 5], [7, 9], [11, 14], [15, 18], [24, 25], [25, 27]]","[OOD_Term, Specific_Term, Specific_Term, Specific_Term, Common_Term, Specific_Term]",htfl,Heart Failure
5,"However, there were no measurable overall improvements for patients admitted with pneumonia or surgical conditions.","[However, ,, there, were, no, measurable, overall, improvements, for, patients, admitted, with, pneumonia, or, surgical, conditions, ., ]","[patients, pneumonia, surgical, conditions]","[[patients], [pneumonia], [surgical], [conditions]]","[[9, 10], [12, 13], [14, 15], [15, 16]]","[Common_Term, Common_Term, Common_Term, Common_Term]",htfl,Heart Failure
6,"Some events, such as pressure ulcers in surgical patients, actually increased despite considerable national attention to these problems.","[Some, events, ,, such, as, pressure, ulcers, in, surgical, patients, ,, actually, increased, despite, considerable, national, attention, to, these, problems, ., ]","[pressure 

In [7]:
from datasets import Dataset, DatasetDict

train_dataset = Dataset.from_pandas(train_dataset, preserve_index=False)
test_dataset = Dataset.from_pandas(test_dataset, preserve_index=False)
validation_dataset = Dataset.from_pandas(validation_dataset, preserve_index=False)

/mnt/raid6/yongchan/miniconda3/envs/ATE/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from datasets import Dataset, DatasetDict

dataset_save_path = 'huggingface'
os.makedirs(dataset_save_path, exist_ok=True)

dataset = DatasetDict({'train': train_dataset, 'validation': validation_dataset, 'test': test_dataset})
dataset.save_to_disk(dataset_save_path)

Saving the dataset (1/1 shards): 100%|██████████| 2432/2432 [00:00<00:00, 34766.57 examples/s]
